In [1]:
import pandas as pd
import numpy as np
 
import tensorflow as tf
 
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
df = pd.read_csv(url)

In [3]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


In [4]:
#Membuat kolom target
# 0 = low; 1 = high
df['target'] = np.where(df['price'] <= 1000, 0, 1)
# Drop un-used columns.
df = df.drop(columns=['price'])

In [5]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [6]:
example_batch = next(iter(train_ds))[0]
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [7]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[0.54]
 [0.3 ]
 [0.32]
 [1.01]
 [0.38]
 [1.11]
 [0.71]
 [0.5 ]
 [1.  ]
 [0.41]]


In [8]:
#bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]]


In [9]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
      'color', ['E', 'I','J','D','H', 'G','F'])
 
color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

[[0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]


In [10]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
      'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[-0.12610956  0.08366698 -0.29570928 -0.27412608  0.02852357  0.05918366]
 [-0.2948951   0.0112644  -0.0731931  -0.53220594 -0.2596468  -0.40819338]
 [-0.12610956  0.08366698 -0.29570928 -0.27412608  0.02852357  0.05918366]
 [-0.525067   -0.15818915 -0.11352478  0.04843055  0.3279273   0.08842979]
 [-0.609454   -0.06141887 -0.04126133  0.46426338 -0.42804515 -0.40530878]
 [-0.10420122 -0.39891586 -0.07964315 -0.36983475  0.24239634  0.34638268]
 [-0.10420122 -0.39891586 -0.07964315 -0.36983475  0.24239634  0.34638268]
 [-0.12610956  0.08366698 -0.29570928 -0.27412608  0.02852357  0.05918366]
 [-0.44646943  0.1541217   0.05444926  0.20761272 -0.366898   -0.19315273]
 [-0.12610956  0.08366698 -0.29570928 -0.27412608  0.02852357  0.05918366]]


In [11]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket(
      'clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]


In [12]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [13]:
#Pilih feature column mana yang akan digunakan
feature_columns = []
# numeric column
for header in ['carat', 'depth', 'x', 'y', 'z']:
  feature_columns.append(feature_column.numeric_column(header))
 
#membuat feature layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [14]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10
1686/1686 [==============================] - 13s 7ms/step - loss: 0.2148 - accuracy: 0.9061 - val_loss: 0.1734 - val_accuracy: 0.9259
Epoch 2/10
1686/1686 [==============================] - 10s 6ms/step - loss: 0.1589 - accuracy: 0.9327 - val_loss: 0.1545 - val_accuracy: 0.9375
Epoch 3/10
1686/1686 [==============================] - 10s 6ms/step - loss: 0.1548 - accuracy: 0.9345 - val_loss: 0.1432 - val_accuracy: 0.9377
Epoch 4/10
1686/1686 [==============================] - 10s 6ms/step - loss: 0.1521 - accuracy: 0.9356 - val_loss: 0.1420 - val_accuracy: 0.9382
Epoch 5/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1491 - accuracy: 0.9358 - val_loss: 0.1528 - val_accuracy: 0.9327
Epoch 6/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1477 - accuracy: 0.9359 - val_loss: 0.1411 - val_accuracy: 0.9378
Epoch 7/10
1686/1686 [==============================] - 7s 4ms/step - loss: 0.1468 - accuracy: 0.9356 - val_loss: 0.1401 - val_accur